In [645]:
import numpy as np
import pandas as pd
import seaborn as sns
%matplotlib inline

from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report,confusion_matrix
from sklearn import preprocessing
from sklearn.preprocessing import MinMaxScaler
import sklearn.metrics as metrics



# Preprocessing and Feature Selection

First put all features in one pandas.DataFrame and add image ids.

In [651]:
#load all csv files needed
df = pd.read_csv('features_all_malte.csv')
df2 = pd.read_csv('features_all_fabian.csv')
ids = pd.read_csv('image_ids.csv')

In [654]:
ids = ids.drop(['dx', 'dx_type', 'age', 'sex', 'localization'], axis = 1)

In [655]:
#add image ids to df2 (features_all_fabian.csv)
merge_df = df2.join(ids, how='outer') 

In [657]:
df = merge_df.set_index('image_id').join(df.set_index('image_id'))

In [658]:
df.head()

,Moment_R_L00,Moment_R_L01,Moment_R_L02,Moment_R_L10,Moment_R_L11,Moment_R_L12,Moment_R_L20,Moment_R_L21,Moment_R_L22,Moment_G_L00,...,nu03,nu12,nu20,nu21,nu30,perimeter,symmetry,dx,validation,malignant
image_id,,,,,,,,,,,,,,,,,,,,,
ISIC_0027419,0.000578,-3.125691,-16.983755,-29.627738,-2.659856,-8.371072,-74.668856,-11.351595,-69.762741,0.000637,...,0.014579,0.023919,0.061852,-0.006439,-0.005693,1179.165643,0.9770473925852277,bkl,1,0
ISIC_0025030,0.000148,6.620587,-57.724036,-5.835733,-6.146287,5.408704,-127.847081,-15.500430,-1.895868,0.000881,...,-0.048263,0.023418,0.125249,0.005926,-0.023605,481.480229,161.07803921906105,bkl,1,0
ISIC_0026769,0.000189,23.620562,-51.707425,-32.906473,10.581304,5.893859,-74.381919,-44.328496,-60.476961,0.000311,...,0.017578,-0.016207,0.202239,0.000484,0.014633,3052.164768,0.9591750731310495,bkl,0,0
ISIC_0025661,0.000311,-16.095788,-77.088365,-13.184954,1.303120,-4.938602,-117.251672,-0.052012,23.886053,0.000204,...,-0.056763,0.019802,0.081058,0.006778,-0.012322,494.492422,0.11282341025738685,bkl,0,0
ISIC_0031633,0.000411,6.413269,50.663407,67.091066,-33.535822,-42.954445,26.455474,-24.612728,-123.307385,0.000022,...,0.002529,-0.007409,0.124105,0.002641,0.002093,2153.106769,0.17135336578585886,bkl,0,0


In [659]:
#sort here already! Needed for export later
df = df.sort_values(by = 'image_id')

In [531]:
#df.to_csv('features_all_together.csv')

In [660]:
#clean up and drop all rows where NAs occur
df = df.dropna()

In [661]:
df_train = df[df['validation'] == 0]
df_test = df[df['validation'] == 1]

# create label DataFrames
y_train = pd.DataFrame(df_train['malignant'] == 1)
y_test = pd.DataFrame(df_test['malignant'] == 1)

In [1807]:
# create feature DataFrames
X_train = df_train.drop(['lesion_id', 'dx', 'validation', 'malignant'], axis = 1)

X_test = df_test.drop(['lesion_id', 'dx', 'validation', 'malignant'], axis = 1)

In [1809]:
#scaling each feature to [0,1]
min_max_scaler = preprocessing.MinMaxScaler()

X_train = pd.DataFrame(min_max_scaler.fit_transform(X_train), columns=X_train.columns)
X_test = pd.DataFrame(min_max_scaler.fit_transform(X_test), columns=X_test.columns)

/anaconda3/lib/python3.6/site-packages/numpy/core/_methods.py:36: RuntimeWarning: overflow encountered in reduce
  return umr_sum(a, axis, dtype, out, keepdims, initial)
/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/data.py:323: DataConversionWarning: Data with input dtype float64, object were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)
/anaconda3/lib/python3.6/site-packages/numpy/core/_methods.py:36: RuntimeWarning: overflow encountered in reduce
  return umr_sum(a, axis, dtype, out, keepdims, initial)
/anaconda3/lib/python3.6/site-packages/numpy/core/_methods.py:36: RuntimeWarning: overflow encountered in reduce
  return umr_sum(a, axis, dtype, out, keepdims, initial)
/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/data.py:323: DataConversionWarning: Data with input dtype float64, object were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)
/anaconda3/lib/python3.6/site-packages/numpy/core/_methods.py:

In [1798]:
#feature selection #1
from sklearn.feature_selection import SelectKBest,f_classif, chi2,mutual_info_classif

select = SelectKBest(f_classif , k=26)
select.fit(X_train, np.ravel(y_train))
names = X_train.columns.values[select.get_support(indices=True)]
X_train = pd.DataFrame(select.transform(X_train))
X_test = pd.DataFrame(select.transform(X_test))

In [1810]:
#feature selection #2
from sklearn.feature_selection import SelectFromModel
from sklearn.ensemble import RandomForestClassifier

select = SelectFromModel(RandomForestClassifier(n_estimators=100, random_state=25)) #random_state to make reproducable
select.fit(X_train, np.ravel(y_train))
names = X_train.columns.values[select.get_support(indices=True)]
X_train = pd.DataFrame(select.transform(X_train))
X_test = pd.DataFrame(select.transform(X_test))

In [1803]:
#feature selection #3
from sklearn.svm import LinearSVC
from sklearn.feature_selection import SelectFromModel

#the smaller C the fewer features selected.
lsvc = LinearSVC(C=0.1, penalty="l1", dual=False).fit(X_train, np.ravel(y_train))
select = SelectFromModel(lsvc)
select.fit(X_train, np.ravel(y_train))
names = X_train.columns.values[select.get_support(indices=True)]
X_train = pd.DataFrame(select.transform(X_train))
X_test = pd.DataFrame(select.transform(X_test))

/anaconda3/lib/python3.6/site-packages/sklearn/svm/base.py:922: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/anaconda3/lib/python3.6/site-packages/sklearn/svm/base.py:922: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


In [1811]:
names

array(['Moment_R_L01', 'Moment_R_L02', 'Moment_R_L20', 'Moment_R_L22',
       'Moment_G_L02', 'Moment_G_L20', 'Moment_B_L02', 'Moment_B_L20',
       'average_red3', 'average_green3', 'average_blue3',
       'area_variance02', 'area_variance03', 'area_variance2',
       'area_variance3', 'average_blue2', 'average_green2',
       'average_red2', 'correlation2', 'energy2', 'nu03', 'nu12', 'nu30',
       'perimeter'], dtype=object)

In [1812]:
X_train = preprocessing.scale(X_train)
X_test  = preprocessing.scale(X_test)

# Training

In [1814]:
model = SVC(kernel='rbf', C=15, gamma='auto', probability = True)

In [1815]:
model.fit(X_train,np.ravel(y_train))

SVC(C=15, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=True, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [1817]:
prediction_probabilities = model.predict_proba(X_test)

In [1819]:
probabs = pd.DataFrame(prediction_probabilities)
probabs.columns = [False, True]
predictions = probabs.idxmax(axis=1)

In [1821]:
confusion = confusion_matrix(y_test,predictions)
print(confusion)


[[1859  159]
 [ 278  215]]


In [ ]:
# check if overfitting the data by looking at training dataset

#prediction_probabilities = model.predict_proba(X_train)
#probabs = pd.DataFrame(prediction_probabilities)
#probabs.columns = [False, True]
#predictions = probabs.idxmax(axis=1)
#confusion = confusion_matrix(y_train,predictions)
#print(confusion)


In [1822]:
# classification metrics (https://stackoverflow.com/a/43331484/8614827)
FP = confusion.sum(axis=0) - np.diag(confusion)  
FN = confusion.sum(axis=1) - np.diag(confusion)
TP = np.diag(confusion)
TN = confusion.sum() - (FP + FN + TP)

# Sensitivity, hit rate, recall, or true positive rate
TPR = TP/(TP+FN)
# Specificity or true negative rate
TNR = TN/(TN+FP) 
# Precision or positive predictive value
PPV = TP/(TP+FP)
# Negative predictive value
NPV = TN/(TN+FN)
# Fall out or false positive rate
FPR = FP/(FP+TN)
# False negative rate
FNR = FN/(TP+FN)
# False discovery rate
FDR = FP/(TP+FP)

# Overall accuracy
ACC = (TP+TN)/(TP+FP+FN+TN)

In [1823]:
# metrics used on MTEC website
print(ACC.mean()) #accuracy
print(TPR.mean()) #sensitivity
print(TNR.mean()) #specificity
print(metrics.f1_score(y_test, predictions, average=None).mean()) #F1-score

0.8259657506969335
0.6786572973059904
0.6786572973059904
0.695394301275405


# Postprocessing and csv Creation

In [1737]:
result = pd.DataFrame(prediction_probabilities)

In [1824]:
image_label = pd.DataFrame(df_test.index)
image_label = image_label.sort_values(by = 'image_id')

In [1652]:
image_label= image_label.reset_index(drop=True)

In [1653]:
to_csv = image_label.join(result, how='outer')

In [1654]:
to_csv.columns = ['image','BEN','MAL']

In [1655]:
to_csv.to_csv('binary_classification', index = False)

In [1656]:
to_csv.head()

,image,BEN,MAL
0,ISIC_0024312,0.819358,0.180642
1,ISIC_0024317,0.797442,0.202558
2,ISIC_0024318,0.945907,0.054093
3,ISIC_0024324,0.813754,0.186246
4,ISIC_0024328,0.938245,0.061755


# Random Forrest

In [462]:
from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier(n_estimators=100)
rfc.fit(X_train, np.ravel(y_train))

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [463]:
prediction_probabilities = rfc.predict_proba(X_test)